In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler; scaler = MinMaxScaler()
from sklearn.model_selection import validation_curve
from sklearn import metrics
from sklearn.dummy import DummyRegressor
%matplotlib notebook

In [2]:
import warnings
warnings.filterwarnings('ignore')

#warnings.filterwarnings(action='once')

On part d'un dataset contenant le taux de mortalité calculé par tranche d'âge et par sexe, pour chaque pays et pour chaque année. 
Les valeurs manquantes des indicateurs ont été remplacées par la dernière année disponible par pays. Le fichier <i>
data_mort_wb_sub.csv</i> est issu du programme "Traitement base de donnée" et il comprend :
    - une séléction d'années : 16 années (2000 - 2015)
    - tous les pays ayant une correpondance entre le fichier mortalité et le fichier worldbank : 99 pays
    - un type de cancer : C33, C34 (cancer du poumon); implicite - la variable type de cancer n'est pas présente dans le dataset
    - une séléction d'indicateurs : 44 choisis parmis ceux avec moins de 25% de valeurs manquantes

<table><tr><th>Category</th><th>Indicator code</th><th>Indicator description</th></tr><tr><td>Education</td><td>SE.PRM.DURS</td><td>Primary education, duration (years)</td></tr><tr><td>Education</td><td>SE.PRE.DURS</td><td>Preprimary education, duration (years)</td></tr><tr><td>Education</td><td>SE.SEC.DURS</td><td>Secondary education, duration (years)</td></tr><tr><td>Environnement</td><td>SP.URB.TOTL.IN.ZS</td><td>Urban population (% of total)</td></tr><tr><td>Environnement</td><td>SP.URB.GROW</td><td>Urban population growth (annual %)</td></tr><tr><td>Environnement</td><td>AG.SRF.TOTL.K2</td><td>Surface area (sq. km)</td></tr><tr><td>Environnement</td><td>EN.POP.DNST</td><td>Population density (people per sq. km of land area)</td></tr><tr><td>Environnement</td><td>EN.ATM.NOXE.KT.CE</td><td>Nitrous oxide emissions (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.METH.KT.CE</td><td>Methane emissions (kt of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>AG.LND.AGRI.K2</td><td>Agricultural land (sq. km)</td></tr><tr><td>Environnement</td><td>EN.ATM.GHGO.KT.CE</td><td>Other greenhouse gas emissions, HFC, PFC and SF6 (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>AG.LND.AGRI.ZS</td><td>Agricultural land (% of land area)</td></tr><tr><td>Environnement</td><td>AG.PRD.FOOD.XD</td><td>Food production index (2004-2006 = 100)</td></tr><tr><td>Environnement</td><td>AG.PRD.LVSK.XD</td><td>Livestock production index (2004-2006 = 100)</td></tr><tr><td>Environnement</td><td>AG.PRD.CROP.XD</td><td>Crop production index (2004-2006 = 100)</td></tr><tr><td>Environnement</td><td>EN.ATM.GHGT.KT.CE</td><td>Total greenhouse gas emissions (kt of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.GF.ZS</td><td>CO2 emissions from gaseous fuel consumption (% of total)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.LF.ZS</td><td>CO2 emissions from liquid fuel consumption (% of total)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.SF.ZS</td><td>CO2 emissions from solid fuel consumption (% of total)</td></tr><tr><td>Environnement</td><td>NY.GDP.FRST.RT.ZS</td><td>Forest rents (% of GDP)</td></tr><tr><td>Environnement</td><td>NY.GDP.MINR.RT.ZS</td><td>Mineral rents (% of GDP)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.LF.KT</td><td>CO2 emissions from liquid fuel consumption (kt)</td></tr><tr><td>Environnement</td><td>EN.ATM.METH.AG.KT.CE</td><td>Agricultural methane emissions (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.METH.EG.KT.CE</td><td>Methane emissions in energy sector (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.NOXE.AG.KT.CE</td><td>Agricultural nitrous oxide emissions (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>EN.ATM.NOXE.EG.KT.CE</td><td>Nitrous oxide emissions in energy sector (thousand metric tons of CO2 equivalent)</td></tr><tr><td>Environnement</td><td>NY.GDP.TOTL.RT.ZS</td><td>Total natural resources rents (% of GDP)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.KT</td><td>CO2 emissions (kt)</td></tr><tr><td>Environnement</td><td>EN.ATM.CO2E.PC</td><td>CO2 emissions (metric tons per capita)</td></tr><tr><td>Health</td><td>SP.POP.GROW</td><td>Population growth (annual %)</td></tr><tr><td>Health</td><td>SP.DYN.CBRT.IN</td><td>Birth rate, crude (per 1,000 people)</td></tr><tr><td>Health</td><td>SP.DYN.CDRT.IN</td><td>Death rate, crude (per 1,000 people)</td></tr><tr><td>Health</td><td>SP.DYN.TFRT.IN</td><td>Fertility rate, total (births per woman)</td></tr><tr><td>Health</td><td>SP.ADO.TFRT</td><td>Adolescent fertility rate (births per 1,000 women ages 15-19)</td></tr><tr><td>Health</td><td>SP.DYN.LE00.FE.IN</td><td>Life expectancy at birth, female (years)</td></tr><tr><td>Health</td><td>SP.DYN.LE00.IN</td><td>Life expectancy at birth, total (years)</td></tr><tr><td>Health</td><td>SP.POP.DPND</td><td>Age dependency ratio (% of working-age population)</td></tr><tr><td>Health</td><td>SP.DYN.AMRT.FE</td><td>Mortality rate, adult, female (per 1,000 female adults)</td></tr><tr><td>Health</td><td>SP.DYN.AMRT.MA</td><td>Mortality rate, adult, male (per 1,000 male adults)</td></tr><tr><td>Infrastructure</td><td>IT.CEL.SETS</td><td>Mobile cellular subscriptions</td></tr><tr><td>Infrastructure</td><td>IT.MLT.MAIN</td><td>Fixed telephone subscriptions</td></tr><tr><td>National_accounts</td><td>NY.ADJ.AEDU.GN.ZS</td><td>Adjusted savings: education expenditure (% of GNI)</td></tr><tr><td>National_accounts</td><td>NY.GDP.MKTP.CD</td><td>GDP (current USD)</td></tr><tr><td>National_accounts</td><td>NY.GDP.PCAP.CD</td><td>GDP per capita (current USD)</td></tr>
</table>

In [2]:
df = pd.read_csv('data_mort_wb_sub.csv', header=0,sep=',', low_memory=False)
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.shape

(37264, 49)

In [4]:
df.head()

,Sex,Age_bucket,Year,Country,Mortality_rate,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SP.POP.GROW,SE.PRM.DURS,AG.SRF.TOTL.K2,...,EN.ATM.CO2E.LF.KT,EN.ATM.METH.AG.KT.CE,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD
0,2,1,2000,Jamaica,0.0,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
1,2,2,2000,Jamaica,0.0,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
2,2,3,2000,Jamaica,0.0,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
3,2,4,2000,Jamaica,0.0,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741
4,2,5,2000,Jamaica,0.0,51.814,1.24118,0.776905,6.0,10990.0,...,9856.896,633.005651,267.408083,361.597144,52.552558,1.762288,10307.937,3.964446,8.929376e+09,3434.249741


In [51]:
print(df['Mortality_rate'].min(),
      df['Mortality_rate'].mean(),
      df['Mortality_rate'].max()
     )
      

0.0 5.08845183750611 142.857142857


In [3]:
#variables explicatives quantitatives
indicators = ['SP.URB.TOTL.IN.ZS',	'SP.URB.GROW',	'SP.POP.GROW',	'SE.PRM.DURS',	'AG.SRF.TOTL.K2',	'SE.PRE.DURS',	'EN.POP.DNST',	'SE.SEC.DURS',	'SP.DYN.CBRT.IN',	'SP.DYN.CDRT.IN',	'NY.ADJ.AEDU.GN.ZS',	'SP.DYN.TFRT.IN',	'SP.ADO.TFRT',	'SP.DYN.LE00.FE.IN',	'SP.DYN.LE00.IN',	'SP.POP.DPND',	'SP.DYN.AMRT.FE',	'SP.DYN.AMRT.MA',	'EN.ATM.NOXE.KT.CE',	'EN.ATM.METH.KT.CE',	'AG.LND.AGRI.K2',	'EN.ATM.GHGO.KT.CE',	'AG.LND.AGRI.ZS',	'AG.PRD.FOOD.XD',	'IT.CEL.SETS',	'AG.PRD.LVSK.XD',	'AG.PRD.CROP.XD',	'IT.MLT.MAIN',	'EN.ATM.GHGT.KT.CE',	'EN.ATM.CO2E.GF.ZS',	'EN.ATM.CO2E.LF.ZS',	'EN.ATM.CO2E.SF.ZS',	'NY.GDP.FRST.RT.ZS',	'NY.GDP.MINR.RT.ZS',	'EN.ATM.CO2E.LF.KT',	'EN.ATM.METH.AG.KT.CE',	'EN.ATM.METH.EG.KT.CE',	'EN.ATM.NOXE.AG.KT.CE',	'EN.ATM.NOXE.EG.KT.CE',	'NY.GDP.TOTL.RT.ZS',	'EN.ATM.CO2E.KT',	'EN.ATM.CO2E.PC',	'NY.GDP.MKTP.CD',	'NY.GDP.PCAP.CD']          

#variables explicatives catégorielles : sex, tranche d'age, pays
df["Country_code"] = df["Country"].astype('category').cat.codes
df["Sex"] = df["Sex"].astype('category')
df["Age_bucket"] = df["Age_bucket"].astype('category')

In [4]:
#variable expliquée : taux de mortalité 
y = df['Mortality_rate']
y.describe()

count    37264.000000
mean         5.088452
std         13.258582
min          0.000000
25%          0.000000
50%          0.126043
75%          3.284422
max        142.857143
Name: Mortality_rate, dtype: float64

<font color="#660066" size=6><u>
SVM pour problème de regression
</font></u>

On compare d'abord les kernels RBF, linéaire et polynomiques sur les features standardisées, avec les paramètres par défaut.

http://scikit-learn.org/stable/modules/svm.html#svm-regression

In [6]:
#découpage des échantillons d'apprentissage et d'évaluation
x = df[indicators + ['Sex', 'Age_bucket', 'Year', 'Country_code']]
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state = 0)
#normalisation des features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
X_train_scaled.size

1341504

In [7]:
%%time
#régression SVM avec kernel rbf avec paramètres optimaux => fit model
from sklearn.svm import SVR
from sklearn import metrics
svr_rbf = SVR(kernel='rbf', C=100, gamma=1, max_iter = -1, cache_size=500)
svr_rbf_fit = svr_rbf.fit(X_train_scaled, y_train) 

print("Précision du régresseur SVC avec kernel rbf sur l'échantillon d'apprentissage : {:.2f}"
     .format(svr_rbf_fit.score(X_train_scaled, y_train)))
print("Précision du régresseur SVC avec kernel rbf sur l'échantillon de test : {:.2f}"
     .format(svr_rbf_fit.score(X_test_scaled, y_test)))

err_train = metrics.mean_squared_error( svr_rbf_fit.predict(X_train_scaled), y_train)**0.5
err_test = metrics.mean_squared_error( svr_rbf_fit.predict(X_test_scaled), y_test)**0.5
print("RMSE du régresseur SVC avec kernel RBF sur l'échantillon d'apprentissage : {:.2f}"
     .format(err_train))
print("RMSE du régresseur SVC avec kernel RBF sur l'échantillon de test : {:.2f}"
     .format(err_test))

Précision du régresseur SVC avec kernel rbf sur l'échantillon d'apprentissage : 0.81
Précision du régresseur SVC avec kernel rbf sur l'échantillon de test : 0.80
RMSE du régresseur SVC avec kernel RBF sur l'échantillon d'apprentissage : 5.68
RMSE du régresseur SVC avec kernel RBF sur l'échantillon de test : 6.06
Wall time: 7min 23s


In [47]:
%%time
Y_pred_orig = svr_rbf_fit.predict(X_test_scaled)

Wall time: 11.3 s


In [48]:
print(Y_pred_orig.min(), 
Y_pred_orig.max(),
Y_pred_orig.mean())

-2.08691238953 79.5558621057 4.467951462


In [49]:
print(y_test.min(), 
y_test.max(),
y_test.mean())

0.0 134.135853094 5.191432140999444


In [30]:
#lecture base de données à prédire
df_pred = pd.read_csv('data_pred_wb.csv', sep=',', low_memory=False)
df_pred = df_pred[df_pred['Age_group'] !='80UP']

In [31]:
df_pred.head()

,Unnamed: 0,Country,Year,Age_group,Sex,Population,area,year,SP.URB.TOTL.IN.ZS,SP.URB.GROW,...,EN.ATM.CO2E.LF.KT,EN.ATM.METH.AG.KT.CE,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD
0,0,Aruba,2000,1,2,3413.0,Aruba,2000,46.717,1.187344,...,2321.211,0.800714,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585
1,1,Aruba,2000,1,1,3565.0,Aruba,2000,46.717,1.187344,...,2321.211,0.800714,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585
2,2,Aruba,2000,2,2,3598.0,Aruba,2000,46.717,1.187344,...,2321.211,0.800714,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585
3,3,Aruba,2000,2,1,3714.0,Aruba,2000,46.717,1.187344,...,2321.211,0.800714,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585
4,4,Aruba,2000,3,2,3375.0,Aruba,2000,46.717,1.187344,...,2321.211,0.800714,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585


In [32]:
#variables explicatives catégorielles : sex, tranche d'age, pays
df_pred["Country_code"] = df_pred["Country"].astype('category').cat.codes
df_pred["Sex"] = df_pred["Sex"].astype('category')
df_pred["Age_group"] = df_pred["Age_group"].astype('category')

In [34]:
#normalisation des features
X_train_pred = scaler.fit_transform(df_pred[indicators + ['Sex', 'Age_group', 'Year', 'Country_code']])


In [35]:
%%time
#utiliser le modèle fit pour prédire
Y_pred = svr_rbf_fit.predict(X_train_pred)

In [36]:
Y_pred

array([  2.61639846,   2.72712284,   2.543181  , ...,  16.43963025,
         9.47029822,  20.01101173])

In [37]:
df_pred2 = df_pred.copy()
df_pred2['prediction'] = Y_pred

In [38]:
df_pred2.head()

,Unnamed: 0,Country,Year,Age_group,Sex,Population,area,year,SP.URB.TOTL.IN.ZS,SP.URB.GROW,...,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,Country_code,prediction
0,0,Aruba,2000,1,2,3413.0,Aruba,2000,46.717,1.187344,...,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585,7,2.616398
1,1,Aruba,2000,1,1,3565.0,Aruba,2000,46.717,1.187344,...,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585,7,2.727123
2,2,Aruba,2000,2,2,3598.0,Aruba,2000,46.717,1.187344,...,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585,7,2.543181
3,3,Aruba,2000,2,1,3714.0,Aruba,2000,46.717,1.187344,...,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585,7,2.867809
4,4,Aruba,2000,3,2,3375.0,Aruba,2000,46.717,1.187344,...,1.030635,0.746749,1.329305,0.000703,2321.211,25.547679,1.873453e+09,20619.56585,7,2.497274


In [40]:
df_pred2.Country.unique()

array(['Aruba', 'Afghanistan', 'Angola', 'Albania', 'United Arab Emirates',
       'Argentina', 'Armenia', 'Antigua and Barbuda', 'Australia',
       'Austria', 'Azerbaijan', 'Burundi', 'Belgium', 'Benin',
       'Burkina Faso', 'Bangladesh', 'Bulgaria', 'Bahrain', 'Bahamas, The',
       'Bosnia and Herzegovina', 'Belarus', 'Belize', 'Bolivia', 'Brazil',
       'Barbados', 'Brunei Darussalam', 'Bhutan', 'Botswana',
       'Central African Republic', 'Canada', 'Switzerland', 'Chile',
       'China', "Cote d'Ivoire", 'Cameroon', 'Congo, Dem. Rep.',
       'Congo, Rep.', 'Colombia', 'Comoros', 'Cabo Verde', 'Costa Rica',
       'Cuba', 'Curacao', 'Cyprus', 'Czech Republic', 'Germany',
       'Djibouti', 'Denmark', 'Dominican Republic', 'Algeria', 'Ecuador',
       'Egypt, Arab Rep.', 'Eritrea', 'Spain', 'Estonia', 'Ethiopia',
       'Finland', 'Fiji', 'France', 'Micronesia, Fed. Sts.', 'Gabon',
       'United Kingdom', 'Georgia', 'Ghana', 'Guinea', 'Gambia, The',
       'Guinea-Bissau', '

In [39]:
df_pred2.describe()

,Unnamed: 0,Year,Population,year,SP.URB.TOTL.IN.ZS,SP.URB.GROW,SP.POP.GROW,SE.PRM.DURS,AG.SRF.TOTL.K2,SE.PRE.DURS,...,EN.ATM.METH.EG.KT.CE,EN.ATM.NOXE.AG.KT.CE,EN.ATM.NOXE.EG.KT.CE,NY.GDP.TOTL.RT.ZS,EN.ATM.CO2E.KT,EN.ATM.CO2E.PC,NY.GDP.MKTP.CD,NY.GDP.PCAP.CD,Country_code,prediction
count,97728.000000,97728.000000,9.772800e+04,97728.000000,97728.000000,97728.000000,97728.000000,97728.000000,9.772800e+04,97728.000000,...,97728.000000,97728.000000,97728.000000,97728.000000,9.772800e+04,97728.000000,9.772800e+04,97728.000000,97728.000000,97728.000000
mean,51916.500000,2007.494434,1.080234e+06,2007.494434,55.268336,2.176262,1.500062,5.696136,7.190571e+05,2.728880,...,13672.347720,10606.256489,1403.518898,8.341714,1.656185e+05,4.734159,3.021664e+11,11145.417518,96.369352,5.230510
std,29975.024451,4.608016,4.550676e+06,4.608016,23.641738,2.025738,1.540600,0.890545,1.947163e+06,0.768471,...,57079.819034,33705.018703,6444.568869,12.759579,7.107065e+05,6.674683,1.239521e+12,16801.852591,55.909978,6.899940
min,0.000000,2000.000000,1.720000e+02,2000.000000,8.246000,-3.462541,-4.075386,3.000000,2.000000e+01,1.000000,...,0.000000,0.000000,0.000000,0.000000,2.566900e+01,0.016908,6.310127e+07,106.016975,0.000000,-2.524502
25%,25957.750000,2003.000000,4.010731e+04,2003.000000,35.591000,0.736963,0.491430,5.000000,2.634000e+04,2.000000,...,261.712171,335.672505,40.070513,0.411516,2.119526e+03,0.584684,4.521425e+09,1071.323339,47.000000,1.393985
50%,51916.500000,2007.000000,1.709325e+05,2007.000000,55.275500,1.972867,1.352793,6.000000,1.303700e+05,3.000000,...,1399.075319,2187.493982,164.153208,2.494133,1.152171e+04,2.273160,1.947205e+10,3772.105302,97.000000,2.943875
75%,77875.250000,2011.000000,6.414690e+05,2011.000000,74.172000,3.424974,2.369273,6.000000,5.490870e+05,3.000000,...,6899.189540,7168.199254,611.034410,10.768693,6.691175e+04,6.462447,1.292501e+11,13342.084998,145.000000,6.396208
max,103833.000000,2015.000000,6.844154e+07,2015.000000,100.000000,17.899614,17.624770,8.000000,1.709825e+07,5.000000,...,738366.843413,375435.620570,82671.322877,89.166112,1.024946e+07,67.452198,1.803665e+13,117507.808593,192.000000,72.324959
